In [1]:
import cv2 as cv
import numpy as np
import epfl_data2 as epfl_data
import matplotlib.pyplot as plt
from skimage.feature import hog as HoG
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
def flip_angle(angle):
    if angle == 180:
        return 0
    elif angle == 0:
        return 180
    else:
        return 360-angle


rotations = 3
bins = 6

data = epfl_data.Data(rotations,bins)

training_images = np.zeros((1327,11016))
training_images_f = np.zeros((1327,11016))
test_images = np.zeros((972,11016))
training_tgts_1 = []
training_tgts_2 = []
training_tgts_3 = []
training_tgts_f1 = []
training_tgts_f2 = []
training_tgts_f3 = []
train_angles = []
train_angles_f = []
test_tgts = []

height = 250
width = 376

for d in range(len(data.samples[0])*2):
    image = cv.imread(data.samples[0][d])
    image = image[28:212, 30:348]
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    f_image = np.fliplr(image)
    if d <= 1326:
        image = HoG(image, orientations=8, pixels_per_cell=(16,16))
        image = normalize(image)
        f_image = HoG(f_image, orientations=8, pixels_per_cell=(16,16))
        f_image = normalize(image)
        training_images[d] = image
        training_tgts_1.append(data.samples[2][d][0])
        training_tgts_2.append(data.samples[2][d][1])
        training_tgts_3.append(data.samples[2][d][2])
        training_images_f[d] = f_image
        training_tgts_f1.append(data.samples[2][d][0])
        training_tgts_f2.append(data.samples[2][d][1])
        training_tgts_f3.append(data.samples[2][d][2])
        
        train_angles.append(np.deg2rad(data.samples[1][d]))
        train_angles_f.append(np.deg2rad(flip_angle(data.samples[1][d])))
    else:
        if d-1326 == 972:
            break
        else:
            image = HoG(image, orientations=8, pixels_per_cell=(16, 16))
            image = normalize(image)
            test_images[d-1326] = image
            test_tgts.append(np.deg2rad(data.samples[1][d]))

train = np.append(training_images, training_images_f,axis=0)
for i in range(len(training_tgts_1)):
    training_tgts_1.append(training_tgts_f1[i])
    training_tgts_2.append(training_tgts_f2[i])
    training_tgts_3.append(training_tgts_f3[i])
for i in range(len(train_angles)):
    train_angles.append(train_angles_f[i])
print('done')

done


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers.convolutional import Conv1D
from keras.models import Model
from keras.losses import categorical_crossentropy as cross

Using TensorFlow backend.


In [4]:
hog_input = Input(shape=(11016,), name='input')
layers = Dense(1000, input_dim=11016, activation='relu')(hog_input)
layers = Dropout(.5)(layers)
layers = Dense(100, activation='relu')(layers)
layers = Dropout(.5)(layers)
layers = Dense(10, activation='relu')(layers)
layers = Dropout(.5)(layers)
outputs = []
for r in range(rotations):
    outputs.append(Dense(bins, activation='softmax', name='output{}'.format(r))(layers))

In [5]:
model = Model(inputs=hog_input, outputs=outputs)
model.compile(optimizer='adam', loss=cross, metrics=['accuracy'])
model.fit(train, 
          [np.asarray(training_tgts_1), np.asarray(training_tgts_2), np.asarray(training_tgts_3)], 
          batch_size=32, epochs=20, verbose=1)

Epoch 1/20
2654/2654 [==============================] - 48s - loss: 5.2961 - output0_loss: 1.7469 - output1_loss: 1.7717 - output2_loss: 1.7775 - output0_acc: 0.2306 - output1_acc: 0.2099 - output2_acc: 0.1873    
Epoch 2/20
2654/2654 [==============================] - 35s - loss: 4.8778 - output0_loss: 1.5770 - output1_loss: 1.6335 - output2_loss: 1.6673 - output0_acc: 0.3433 - output1_acc: 0.3026 - output2_acc: 0.2735    
Epoch 3/20
2654/2654 [==============================] - 36s - loss: 4.2049 - output0_loss: 1.3616 - output1_loss: 1.3826 - output2_loss: 1.4607 - output0_acc: 0.4435 - output1_acc: 0.4439 - output2_acc: 0.3775    
Epoch 4/20
2654/2654 [==============================] - 34s - loss: 3.6402 - output0_loss: 1.2208 - output1_loss: 1.1652 - output2_loss: 1.2543 - output0_acc: 0.5075 - output1_acc: 0.5215 - output2_acc: 0.4872    
Epoch 5/20
2654/2654 [==============================] - 35s - loss: 3.2013 - output0_loss: 1.0813 - output1_loss: 1.0357 - output2_loss: 1.0843 

In [6]:
second_round = model.predict(train)

In [7]:
second_array = np.asarray(second_round)
second_array = np.moveaxis(second_array, 1,0)
flat_second = second_array.reshape((-1, 18))

In [13]:
vonMises = Sequential()
vonMises.add(Dense(18, input_dim=18, activation='relu'))
vonMises.add(Dropout(.5))
vonMises.add(Dense(15, activation='relu'))
vonMises.add(Dropout(.5))
vonMises.add(Dense(10, activation='relu'))
vonMises.add(Dropout(.5))
vonMises.add(Dense(5, activation='relu'))
vonMises.add(Dropout(.5))
vonMises.add(Dense(5, activation='relu'))
vonMises.add(Dropout(.5))
vonMises.add(Dense(1))

In [26]:
vonMises.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
vonMises.fit(flat_second, 
             np.asarray(train_angles),
             batch_size=32, epochs=50, verbose=1)

Epoch 1/50
2654/2654 [==============================] - 1s - loss: 3.4605 - acc: 0.0000e+00     
Epoch 2/50
2654/2654 [==============================] - 0s - loss: 3.4500 - acc: 0.0000e+00     
Epoch 3/50
2654/2654 [==============================] - 0s - loss: 3.4402 - acc: 0.0000e+00     
Epoch 4/50
2654/2654 [==============================] - 0s - loss: 3.4238 - acc: 0.0000e+00     
Epoch 5/50
2654/2654 [==============================] - 0s - loss: 3.3958 - acc: 0.0000e+00     
Epoch 6/50
2654/2654 [==============================] - 0s - loss: 3.3925 - acc: 0.0000e+00     
Epoch 7/50
2654/2654 [==============================] - 0s - loss: 3.3561 - acc: 0.0000e+00     
Epoch 8/50
2654/2654 [==============================] - 0s - loss: 3.4248 - acc: 0.0000e+00     
Epoch 9/50
2654/2654 [==============================] - 0s - loss: 3.3519 - acc: 0.0000e+00     
Epoch 10/50
2654/2654 [==============================] - 0s - loss: 3.3400 - acc: 0.0000e+00     
Epoch 11/50
2654/2654 [======

In [27]:
outputTest = model.predict(test_images[:-1])
outputTest = np.asarray(outputTest)
outputTest = np.moveaxis(outputTest, 1,0)
outputTest = outputTest.reshape((-1,18))
print(len(outputTest))
print(len(test_tgts))
score2 = vonMises.predict(outputTest)



print("PREDICTIONS 1-10:", score2[:10].T)
print('\n')
print("ACTUAL 1-10:", test_tgts[:10])

971
971
PREDICTIONS 1-10: [[ 3.11704159  3.11704159  3.11704159  3.11704159  3.11704159  3.11704159
   3.11704159  3.11704159  3.11704159  3.11704159]]


ACTUAL 1-10: [0.68815839078633589, 0.62831853071795862, 0.5684786706495818, 0.50863881058120453, 0.44879895051282775, 0.38895909044445093, 0.32911923037607366, 0.32911923037607366, 0.20943951023931956, 0.14959965017094273]
